In [ ]:
import json
from data_preprocessing import preprocessing, DUMMY_PENALTY

def get_pair_probability(pair, truth_booths, truth_booths_sold):
    """
    Ermittelt für ein Paar die Wahrscheinlichkeit:
      - Falls das Paar in den truth_booths mit is_match == True vorkommt,
        wird 1 zurückgegeben.
      - Falls das Paar in den truth_booths mit is_match == False vorkommt,
        wird 0 zurückgegeben.
      - Falls das Paar in den truth_booths_sold vorkommt, wird ebenfalls 1
        angenommen.
      - Ansonsten wird None zurückgegeben (keine Information).
    """
    # Prüfe truth_booths
    for tb in truth_booths:
        if tb["man"] == pair["man"] and tb["woman"] == pair["woman"]:
            return 1 if tb.get("is_match", False) else 0

    # Prüfe truth_booths_sold
    for ts in truth_booths_sold:
        if ts["man"] == pair["man"] and ts["woman"] == pair["woman"]:
            return 1

    return None

def calculate_ceremony_weight(ceremony, truth_booths, truth_booths_sold):
    """
    Berechnet das Gewicht für eine Matching Ceremony:
      1. Startet mit unbekannte_paare = 10 und übernimmt den score der Nacht.
      2. Für jedes Paar in ceremony["pairs"]:
           - Wenn das Paar eine Wahrscheinlichkeit von 1 hat, werden unbekannte_paare
             und score um 1 reduziert.
           - Wenn das Paar eine Wahrscheinlichkeit von 0 hat, wird nur unbekannte_paare um 1
             reduziert.
      3. Das finale Gewicht ist: unbekannte_paare - score.
         Ein niedriger Wert signalisiert eine höhere Gewichtung.
         
    Zusätzlich wird der aktuelle Wert der unbekannten Paare (und der final reduzierte Score)
    zurückgegeben, um die Zwischenstände kontrollieren zu können.
    """
    unbekannte_paare = 10
    score = ceremony["score"]

    for pair in ceremony.get("pairs", []):
        prob = get_pair_probability(pair, truth_booths, truth_booths_sold)
        if prob == 1:
            unbekannte_paare -= 1
            score -= 1
        elif prob == 0:
            unbekannte_paare -= 1
        # Falls prob None ist, bleibt alles unverändert.

    gewicht = unbekannte_paare - score
    return gewicht, unbekannte_paare, score

def assign_ceremony_probabilities(data):
    """
    Berechnet für jede Matching Ceremony das Gewicht und leitet daraus
    Wahrscheinlichkeiten ab.
    
    Dabei wird:
      - Das Gewicht jeder Ceremony berechnet.
      - Zusätzlich werden die unbekannten Paare und der finale Score ausgegeben.
      - Anschließend wird transformiert:
           transformiertes Gewicht = (max_gewicht - gewicht) + 1,
        sodass Ceremonies mit niedrigerem Gewicht (höhere Gewichtung) einen
        größeren Anteil erhalten.
      - Die Wahrscheinlichkeiten werden normiert, sodass ihre Summe 1 ergibt.
    """
    ceremonies = data["match_ceremonies"]
    truth_booths = data["truth_booths"]
    truth_booths_sold = data["truth_booths_sold"]

    debug_info = []
    gewichte = []
    for i, c in enumerate(ceremonies):
        gewicht, unbekannte_paare, final_score = calculate_ceremony_weight(c, truth_booths, truth_booths_sold)
        gewichte.append(gewicht)
        debug_info.append({
            "ceremony_index": i,
            "unbekannte_paare": unbekannte_paare,
            "final_score": final_score,
            "gewicht": gewicht
        })
    
    print("Debug Info pro Ceremony:")
    for info in debug_info:
        print(f"Ceremony {info['ceremony_index'] + 1}: unbekannte_paare = {info['unbekannte_paare']}, final_score = {info['final_score']}, gewicht = {info['gewicht']}")

    # Transformation: Damit Ceremonies mit niedrigerem Gewicht (höhere Gewichtung) mehr Gewicht erhalten,
    # wird transformiert: transformiertes Gewicht = (max_gewicht - gewicht) + 1.
    max_gewicht = max(gewichte)
    transformierte_gewichte = [(max_gewicht - g) + 1 for g in gewichte]
    print("Transformierte Gewichte:", transformierte_gewichte)
    
    # Normalisiere zu Wahrscheinlichkeiten
    total = sum(transformierte_gewichte)
    wahrscheinlichkeiten = [tw / total for tw in transformierte_gewichte]
    return wahrscheinlichkeiten

def load_data_from_json(filename):
    """
    Lädt die Daten aus der angegebenen JSON-Datei.
    """
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

if __name__ == "__main__":
    SEASON = "Season_6.json"
    season = SEASON
    probabilities, fixed, participants, ceremonies = preprocessing(season)
    
    # Berechne die Wahrscheinlichkeiten für jede Matching Ceremony
    ceremony_probabilities = assign_ceremony_probabilities(data)
    
    print("\nZugewiesene Wahrscheinlichkeiten pro Ceremony:")
    for i, prob in enumerate(ceremony_probabilities, start=1):
        print(f"Ceremony {i}: {prob:.4f}")


In [ ]:
import json

def build_perfect_matches(truth_booths):
    """
    Baut aus den truth_booths (mit is_match True)
    ein Dictionary mit den perfekten Matches für Männer und das inverse Mapping für Frauen.
    """
    perfect_matches = {}
    for tb in truth_booths:
        if tb.get("is_match", False):
            perfect_matches[tb["man"]] = tb["woman"]
    perfect_matches_women = {woman: man for man, woman in perfect_matches.items()}
    return perfect_matches, perfect_matches_women

def get_pair_probability(pair, truth_booths, perfect_matches, perfect_matches_women):
    """
    Ermittelt für ein Paar die Wahrscheinlichkeit:
    
      1. Falls das Paar explizit in truth_booths gefunden wird:
           - Ist is_match True → Rückgabe 1
           - Ist is_match False → Rückgabe 0
      2. Falls ein perfektes Match bereits bekannt ist:
           - Hat der Mann ein perfektes Match und die Frau ist nicht dieses Match → Rückgabe 0
           - Hat die Frau ein perfektes Match und der Mann ist nicht dieses Match → Rückgabe 0
      3. Andernfalls: Keine Information (None)
    """
    # Prüfe explizit in truth_booths
    for tb in truth_booths:
        if tb["man"] == pair["man"] and tb["woman"] == pair["woman"]:
            return 1 if tb.get("is_match", False) else 0

    man = pair["man"]
    woman = pair["woman"]
    # Falls der Mann bereits einen perfekten Partner hat und die Frau nicht dieser ist, dann 0.
    if man in perfect_matches and perfect_matches[man] != woman:
        return 0
    # Analog: Falls die Frau bereits einen perfekten Partner hat und der Mann nicht dieser ist, dann 0.
    if woman in perfect_matches_women and perfect_matches_women[woman] != man:
        return 0

    return None

def calculate_ceremony_weight(ceremony, truth_booths, perfect_matches, perfect_matches_women):
    """
    Berechnet das finale Gewicht für eine Matching Ceremony:
      1. Startet mit unbekannte_paare = 10, und übernimmt den Score (ohne diesen zu verändern).
      2. Für jedes Paar in ceremony["pairs"]:
           - Falls das Paar bekannt ist (d.h. get_pair_probability gibt 0 oder 1 zurück),
             wird unbekannte_paare um 1 reduziert.
      3. Am Ende wird der Score vom reduzierten Wert abgezogen.
      4. Negative Ergebnisse werden auf 0 gesetzt.
    
    Rückgabe:
      - final_unbekannte_paare: Endwert nach Abzug des Scores.
      - raw_unbekannte_paare: Wert nach Reduktion der bekannten Paare (vor Score-Abzug).
      - score: Ursprünglicher Score der Ceremony.
    """
    raw_unbekannte_paare = 10
    score = ceremony["score"]

    for pair in ceremony.get("pairs", []):
        prob = get_pair_probability(pair, truth_booths, perfect_matches, perfect_matches_women)
        if prob in [0, 1]:
            raw_unbekannte_paare -= 1

    final_unbekannte_paare = raw_unbekannte_paare - score
    if final_unbekannte_paare < 0:
        final_unbekannte_paare = 0

    return final_unbekannte_paare, raw_unbekannte_paare, score

def assign_ceremony_probabilities(data):
    """
    Berechnet für jede Matching Ceremony das finale Gewicht (basierend auf den reduzierten unbekannten Paaren)
    und leitet daraus Wahrscheinlichkeiten ab.
    
    Transformation:
      transformiertes Gewicht = (max(final_weight) - final_weight) + 1,
    sodass Ceremonies mit einem niedrigeren final_weight (also höherer Gewichtung) einen höheren Anteil erhalten.
    Anschließend werden die Werte normiert, sodass ihre Summe 1 ergibt.
    """
    ceremonies = data["match_ceremonies"]
    truth_booths = data["truth_booths"]

    perfect_matches, perfect_matches_women = build_perfect_matches(truth_booths)

    debug_info = []
    final_weights = []
    for i, c in enumerate(ceremonies):
        final_weight, raw_unbekannte, score = calculate_ceremony_weight(c, truth_booths, perfect_matches, perfect_matches_women)
        final_weights.append(final_weight)
        debug_info.append({
            "ceremony_index": i,
            "raw_unbekannte_paare": raw_unbekannte,
            "score": score,
            "final_weight": final_weight
        })
    
    print("Debug Info pro Ceremony:")
    for info in debug_info:
        print(f"Ceremony {info['ceremony_index'] + 1}: raw_unbekannte_paare = {info['raw_unbekannte_paare']}, score = {info['score']}, final_weight = {info['final_weight']}")

    max_weight = max(final_weights)
    transformed_weights = [(max_weight - w) + 1 for w in final_weights]
    print("Transformierte Gewichte:", transformed_weights)
    
    total = sum(transformed_weights)
    probabilities = [tw / total for tw in transformed_weights]
    return probabilities

def load_data_from_json(filename):
    """
    Lädt die Daten aus der angegebenen JSON-Datei.
    """
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

if __name__ == "__main__":
    # Passe den Dateinamen an, z.B. 'deine_datei.json'
    SEASON = "Season_6.json"
    path = f".\\src\\normal\\"

    filename = SEASON
    
    data = load_data_from_json(path + filename)
    
    ceremony_probabilities = assign_ceremony_probabilities(data)
    
    print("\nZugewiesene Wahrscheinlichkeiten pro Ceremony:")
    for i, prob in enumerate(ceremony_probabilities, start=1):
        print(f"Ceremony {i}: {prob:.4f}")


In [1]:
import json

def build_perfect_matches(truth_booths):
    """
    Baut aus den truth_booths (mit is_match True) ein Dictionary,
    das angibt, welches perfekte Match für einen Mann bekannt ist,
    sowie ein inverses Mapping für Frauen.
    """
    perfect_matches = {}
    for tb in truth_booths:
        if tb.get("is_match", False):
            perfect_matches[tb["man"]] = tb["woman"]
    perfect_matches_women = {woman: man for man, woman in perfect_matches.items()}
    return perfect_matches, perfect_matches_women

def get_pair_probability(pair, truth_booths, perfect_matches, perfect_matches_women):
    """
    Ermittelt für ein Paar die Wahrscheinlichkeit:
    
    1. Falls das Paar explizit in truth_booths gefunden wird:
         - Ist is_match True → Rückgabe 1
         - Ist is_match False → Rückgabe 0
    2. Falls entweder der Mann oder die Frau bereits ein perfektes Match haben
       und das aktuelle Paar von diesem abweicht, wird 0 zurückgegeben.
       (Hier wird mit OR gearbeitet, sodass ein Paar nicht doppelt abgezogen wird.)
    3. Andernfalls: Rückgabe None (keine Information).
    """
    # Explizite Prüfung in truth_booths:
    for tb in truth_booths:
        if tb["man"] == pair["man"] and tb["woman"] == pair["woman"]:
            return 1 if tb.get("is_match", False) else 0
    
    man = pair["man"]
    woman = pair["woman"]
    if (man in perfect_matches and perfect_matches[man] != woman) or \
       (woman in perfect_matches_women and perfect_matches_women[woman] != man):
        return 0
    
    return None

def calculate_ceremony_weight(ceremony, truth_booths, perfect_matches, perfect_matches_women):
    """
    Berechnet das finale Verhältnis (final_weight) für eine Ceremony.
    
    Vorgehen:
      1. Setze raw_unknown auf 10. Falls ein "no_pair:"-Eintrag existiert, ziehe 1 ab.
      2. Iteriere über alle Paare der Ceremony:
           - Falls get_pair_probability einen Wert (0 oder 1) liefert, wird raw_unknown um 1 reduziert.
           - Gleichzeitig wird bei einem Ergebnis 1 (perfektes Match) ein Zähler erhöht.
      3. Der ursprüngliche Score wird um die Anzahl der bekannten perfekten Matches reduziert:
             adjusted_score = score - perfect_match_count.
      4. Das finale Verhältnis (final_weight) wird als raw_unknown / adjusted_score berechnet 
         (falls adjusted_score > 0 ist, ansonsten setzen wir einen hohen Wert, um diese Ceremony niedrig zu gewichten).
    """
    raw_unknown = 10

    perfect_match_count = 0
    for pair in ceremony.get("pairs", []):
        prob = get_pair_probability(pair, truth_booths, perfect_matches, perfect_matches_women)
        if prob is not None:
            raw_unknown -= 1
            if prob == 1:
                perfect_match_count += 1

    score = ceremony["score"]
    adjusted_score = score - perfect_match_count
    if adjusted_score < 0:
        adjusted_score = 0

    if adjusted_score > 0:
        final_weight = raw_unknown / adjusted_score
    else:
        final_weight = raw_unknown + 100  # hoher Wert, falls kein Potenzial mehr vorhanden.

    return final_weight, raw_unknown, score, perfect_match_count, adjusted_score

def assign_ceremony_probabilities(data):
    """
    Berechnet für jede Matching Ceremony das Verhältnis aus den verbleibenden unbekannten Paaren 
    und den potenziellen perfekten Matches (adjusted_score) und leitet daraus Wahrscheinlichkeiten ab.
    
    Transformation:
      transformiertes Gewicht = (max(final_weight) - final_weight) + 1,
    sodass Ceremonies mit einem niedrigeren final_weight (besseres Verhältnis) einen höheren Anteil erhalten.
    Anschließend werden die Werte normiert, sodass ihre Summe 1 ergibt.
    """
    ceremonies = data["match_ceremonies"]
    truth_booths = data["truth_booths"]

    perfect_matches, perfect_matches_women = build_perfect_matches(truth_booths)

    debug_info = []
    final_weights = []
    for i, c in enumerate(ceremonies):
        final_weight, raw_unknown, score, perfect_match_count, adjusted_score = calculate_ceremony_weight(
            c, truth_booths, perfect_matches, perfect_matches_women
        )
        final_weights.append(final_weight)
        debug_info.append({
            "ceremony_index": i,
            "raw_unknown": raw_unknown,
            "score": score,
            "perfect_match_count": perfect_match_count,
            "adjusted_score": adjusted_score,
            "final_weight": final_weight
        })
    
    print("Debug Info pro Ceremony:")
    for info in debug_info:
        print(f"Ceremony {info['ceremony_index']+1}: raw_unknown = {info['raw_unknown']}, "
              f"score = {info['score']}, perfect_match_count = {info['perfect_match_count']}, "
              f"adjusted_score = {info['adjusted_score']}, final_weight = {info['final_weight']:.4f}")

    max_weight = max(final_weights)
    transformed_weights = [(max_weight - w) + 1 for w in final_weights]
    print("Transformierte Gewichte:", transformed_weights)
    
    total = sum(transformed_weights)
    probabilities = [tw / total for tw in transformed_weights]
    return probabilities

def load_data_from_json(filename):
    """
    Lädt die Daten aus der angegebenen JSON-Datei.
    """
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

if __name__ == "__main__":
    # Passe den Dateinamen an, z.B. 'deine_datei.json'
    SEASON = "Season_6.json"
    path = f".\\src\\normal\\"

    filename = SEASON
    
    data = load_data_from_json(path + filename)
    
    ceremony_probabilities = assign_ceremony_probabilities(data)
    
    print("\nZugewiesene Wahrscheinlichkeiten pro Ceremony:")
    for i, prob in enumerate(ceremony_probabilities, start=1):
        print(f"Ceremony {i}: {prob:.4f}")


Debug Info pro Ceremony:
Ceremony 1: raw_unknown = 5, score = 2, perfect_match_count = 1, adjusted_score = 1, final_weight = 5.0000
Ceremony 2: raw_unknown = 5, score = 3, perfect_match_count = 1, adjusted_score = 2, final_weight = 2.5000
Ceremony 3: raw_unknown = 6, score = 4, perfect_match_count = 1, adjusted_score = 3, final_weight = 2.0000
Ceremony 4: raw_unknown = 6, score = 4, perfect_match_count = 2, adjusted_score = 2, final_weight = 3.0000
Ceremony 5: raw_unknown = 7, score = 4, perfect_match_count = 3, adjusted_score = 1, final_weight = 7.0000
Ceremony 6: raw_unknown = 7, score = 5, perfect_match_count = 3, adjusted_score = 2, final_weight = 3.5000
Ceremony 7: raw_unknown = 6, score = 6, perfect_match_count = 2, adjusted_score = 4, final_weight = 1.5000
Transformierte Gewichte: [3.0, 5.5, 6.0, 5.0, 1.0, 4.5, 6.5]

Zugewiesene Wahrscheinlichkeiten pro Ceremony:
Ceremony 1: 0.0952
Ceremony 2: 0.1746
Ceremony 3: 0.1905
Ceremony 4: 0.1587
Ceremony 5: 0.0317
Ceremony 6: 0.1429
Cer

In [2]:
import json
import numpy as np
from scipy.optimize import linear_sum_assignment

# --- Vorherige Funktionen, die wir bereits definiert haben ---
def build_perfect_matches(truth_booths):
    perfect_matches = {}
    for tb in truth_booths:
        if tb.get("is_match", False):
            perfect_matches[tb["man"]] = tb["woman"]
    perfect_matches_women = {woman: man for man, woman in perfect_matches.items()}
    return perfect_matches, perfect_matches_women

def get_pair_probability(pair, truth_booths, perfect_matches, perfect_matches_women):
    for tb in truth_booths:
        if tb["man"] == pair["man"] and tb["woman"] == pair["woman"]:
            return 1 if tb.get("is_match", False) else 0
    man = pair["man"]
    woman = pair["woman"]
    if (man in perfect_matches and perfect_matches[man] != woman) or \
       (woman in perfect_matches_women and perfect_matches_women[woman] != man):
        return 0
    return None

def calculate_ceremony_weight(ceremony, truth_booths, perfect_matches, perfect_matches_women):
    raw_unknown = 10
    if "no_pair:" in ceremony and ceremony["no_pair:"]:
        raw_unknown -= 1
    perfect_match_count = 0
    for pair in ceremony.get("pairs", []):
        prob = get_pair_probability(pair, truth_booths, perfect_matches, perfect_matches_women)
        if prob is not None:
            raw_unknown -= 1
            if prob == 1:
                perfect_match_count += 1
    score = ceremony["score"]
    adjusted_score = score - perfect_match_count
    if adjusted_score < 0:
        adjusted_score = 0
    if adjusted_score > 0:
        final_weight = raw_unknown / adjusted_score
    else:
        final_weight = raw_unknown + 100  # hoher Wert, falls kein Potenzial mehr vorhanden.
    return final_weight, raw_unknown, score, perfect_match_count, adjusted_score

def assign_ceremony_probabilities(data):
    ceremonies = data["match_ceremonies"]
    truth_booths = data["truth_booths"]
    perfect_matches, perfect_matches_women = build_perfect_matches(truth_booths)
    final_weights = []
    for c in ceremonies:
        final_weight, _, _, _, _ = calculate_ceremony_weight(c, truth_booths, perfect_matches, perfect_matches_women)
        final_weights.append(final_weight)
    max_weight = max(final_weights)
    transformed_weights = [(max_weight - w) + 1 for w in final_weights]
    total = sum(transformed_weights)
    probabilities = [tw / total for tw in transformed_weights]
    return probabilities

# --- Neue Funktionen zum Aggregieren der Paar-Hinweise und zur Bestimmung der perfekten Matches ---

def aggregate_pair_scores(data, ceremony_probabilities, perfect_matches, perfect_matches_women):
    """
    Aggregiert Hinweise aus allen Ceremonies:
      - Für unsichere Paare (get_pair_probability gibt None) wird das Ceremony-Gewicht addiert.
      - Für bestätigte perfekte Matches (1) wird ein hoher Bonus (z. B. 1000) zugewiesen.
      - Für explizit ausgeschlossene Paare (0) wird ein sehr negativer Score (z. B. -1000) zugewiesen.
    """
    pair_scores = {}  # Schlüssel: (man, woman)
    ceremonies = data["match_ceremonies"]
    truth_booths = data["truth_booths"]
    
    for i, ceremony in enumerate(ceremonies):
        weight = ceremony_probabilities[i]
        for pair in ceremony.get("pairs", []):
            prob = get_pair_probability(pair, truth_booths, perfect_matches, perfect_matches_women)
            key = (pair["man"], pair["woman"])
            if prob == 0:
                # Sobald ein Paar ausgeschlossen wird, überschreiben wir den Score mit -1000.
                pair_scores[key] = -1000
            elif prob == 1:
                # Bestätigtes perfektes Match: hoher Bonus
                pair_scores[key] = 1000
            elif prob is None:
                # Unsichere Paare: Summe der Gewichte aus den Ceremonies
                pair_scores[key] = pair_scores.get(key, 0) + weight
    return pair_scores


def get_all_possible_pairs(data):
    """
    Gibt alle möglichen Kombinationen (man, woman) aus den Teilnehmern zurück.
    """
    men = data["participants"]["men"]
    women = data["participants"]["women"]
    return [(m, w) for m in men for w in women]

def find_perfect_matches(data, ceremony_probabilities):
    """
    Nutzt die aggregierten Hinweise (gewichtet mit den Ceremony-Gewichten),
    um mit dem ungarischen Algorithmus eine Zuordnung (perfekte Matches) zu ermitteln.
    """
    truth_booths = data["truth_booths"]
    perfect_matches, perfect_matches_women = build_perfect_matches(truth_booths)
    
    # Aggregiere Paar-Scores aus allen Ceremonies
    pair_scores = aggregate_pair_scores(data, ceremony_probabilities, perfect_matches, perfect_matches_women)
    
    men = data["participants"]["men"]
    women = data["participants"]["women"]
    cost_matrix = np.zeros((len(men), len(women)))
    
    # Fülle die Kostenmatrix: Da der ungarische Algorithmus minimiert, verwenden wir den negativen Score.
    for i, m in enumerate(men):
        for j, w in enumerate(women):
            score = pair_scores.get((m, w), 0)
            cost_matrix[i, j] = -score  # Höhere Scores werden zu niedrigeren Kosten.
    
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    assignment = {}
    for i, j in zip(row_ind, col_ind):
        assignment[men[i]] = women[j]
    return assignment

def load_data_from_json(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

# --- Anwendung ---
if __name__ == "__main__":
    SEASON = "Season_6.json"
    path = f".\\src\\normal\\"

    filename = SEASON
    
    data = load_data_from_json(path + filename)
    
    # Zunächst die Ceremony-Gewichte berechnen:
    ceremony_probabilities = assign_ceremony_probabilities(data)
    print("Zugewiesene Wahrscheinlichkeiten pro Ceremony:")
    for i, prob in enumerate(ceremony_probabilities, start=1):
        print(f"Ceremony {i}: {prob:.4f}")
    
    # Anschließend die perfekten Matches ermitteln:
    perfect_assignment = find_perfect_matches(data, ceremony_probabilities)
    print("\nErmittelte perfekte Matches:")
    for man, woman in perfect_assignment.items():
        print(f"{man} - {woman}")


Zugewiesene Wahrscheinlichkeiten pro Ceremony:
Ceremony 1: 0.1050
Ceremony 2: 0.1749
Ceremony 3: 0.1866
Ceremony 4: 0.1574
Ceremony 5: 0.0350
Ceremony 6: 0.1399
Ceremony 7: 0.2012

Ermittelte perfekte Matches:
danish - nadja
dino - deisy
dion - chiara
enes - selina
josh - ina
joshua - joanna
kaan - sophia
levin - camelia
sinan - nasti
tano - tori
